**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
library(RobustRankAggreg)
show_env()

You are working on        Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
REPO DIRECTORY (FD_REPO): /data/reddylab/Kuei/repo 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/work 
DATA DIRECTORY (FD_DATA): /data/reddylab/Kuei/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log 
PROJECT REF     (FD_REF): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/references 



**Check: region directory**

In [2]:
txt_fdiry = file.path(FD_RES, "region_coverage_fcc_score")

vec = dir(txt_fdiry)
for (txt in vec){cat(txt, "\n")}

atac_ENCFF333TAT 
atac_ENCFF558BLC 
atac_ENCFF925CYR 
atac_ENCFF948AFM 
dnase_ENCFF185XRG 
dnase_ENCFF274YGF 
fcc_astarr_macs_input_overlap 
fcc_astarr_macs_input_union 


**Check: assay directory**

In [3]:
txt_region = "fcc_astarr_macs_input_union"
txt_fdiry  = file.path(FD_RES, "region_coverage_fcc_score", txt_region)
vec = dir(txt_fdiry)
for (txt in vec) {cat(txt, "\n")}

CRISPRi_FlowFISH_K562_Riley_JinWoo 
CRISPRi_Growth_K562_Gersbach_JinWoo 
MPRA_Lenti_K562_Nadav_Vikram 
MPRA_Tiling_K562_Tewhey_Hannah 
STARR_ATAC_K562_Reddy_KS274 
STARR_ATAC_K562_Reddy_KS91 
STARR_ATAC_K562_Reddy_KSMerge 
STARR_WHG_K562_Reddy_A001 
summary 


**Set global variable**

In [4]:
## init: region folder
vec = c("fcc_astarr_macs_input_overlap", "fcc_astarr_macs_input_union")
names(vec) = vec

VEC_TXT_FOLDER = vec
for (txt in vec) {cat(txt, "\n")}

fcc_astarr_macs_input_overlap 
fcc_astarr_macs_input_union 


**Check input files**

In [5]:
txt_folder = VEC_TXT_FOLDER[1]
txt_fdiry  = file.path(FD_RES, "region_coverage_fcc_score", txt_folder, "summary")
txt_fname  = "region.coverage.score.*.tsv"
txt_fglob  = file.path(txt_fdiry, txt_fname)

vec = Sys.glob(txt_fglob)
for (txt in vec) {cat(txt, "\n")}

/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc_score/fcc_astarr_macs_input_overlap/summary/region.coverage.score.atac.tsv 
/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc_score/fcc_astarr_macs_input_overlap/summary/region.coverage.score.fcc.final.tsv 
/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc_score/fcc_astarr_macs_input_overlap/summary/region.coverage.score.fcc.test.tsv 
/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc_score/fcc_astarr_macs_input_overlap/summary/region.coverage.score.fcc.total.tsv 


## Import data

In [6]:
lst = lapply(VEC_TXT_FOLDER, function(txt_folder){
    ### set file directory
    txt_fdiry  = file.path(
        FD_RES, "region_coverage_fcc_score", 
        txt_folder, 
        "summary"
    )
    txt_fname = "region.coverage.score.fcc.final.tsv"
    txt_fpath = file.path(txt_fdiry, txt_fname)

    ### read table
    dat = read_tsv(txt_fpath, show_col_types = FALSE)
    return(dat)
})

### assign and show
lst_dat_region_score_import = lst

res = lapply(lst, dim)
print(res)

dat = lst[[1]]
head(dat, 3)

$fcc_astarr_macs_input_overlap
[1] 432928     11

$fcc_astarr_macs_input_union
[1] 639718     11



Chrom,ChromStart,ChromEnd,Region,Score,Zcore,Score_Label,Zcore_Label,Assay_Folder,Assay_Group,Assay_Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
chr1,10038,10405,chr1:10038-10405,-2.23485265,-3.5573621,Log2FC,Scale(Log2FC),STARR_ATAC_K562_Reddy_KS91,ASTARR_KS91,ASTARR
chr1,14282,14614,chr1:14282-14614,0.03179044,0.0361388,Log2FC,Scale(Log2FC),STARR_ATAC_K562_Reddy_KS91,ASTARR_KS91,ASTARR
chr1,16025,16338,chr1:16025-16338,-1.33934060,-2.1376311,Log2FC,Scale(Log2FC),STARR_ATAC_K562_Reddy_KS91,ASTARR_KS91,ASTARR


**Check: Assay count**

In [7]:
lst = lst_dat_region_score_import 
lst = lapply(lst, function(dat){
    dat = dat %>% 
        dplyr::group_by(Assay_Group, Assay_Label) %>%
        dplyr::summarise(Freq = n(), .groups = "drop")
    return(dat)
})

dat = bind_rows(lst, .id = "Folder")
dat = dat %>% tidyr::spread(Folder, Freq)
fun_display_table(dat)

Assay_Group,Assay_Label,fcc_astarr_macs_input_overlap,fcc_astarr_macs_input_union
ASTARR_KS91,ASTARR,150040,246850
CRISPRi-Growth,CRISPRi-Growth,72743,80288
CRISPRi-HCRFF,CRISPRi-HCRFF,925,1330
LMPRA,LMPRA,61478,68497
TMPRA,TMPRA,1148,1722
WSTARR,WSTARR,146594,241031


**Check: Missing values**

In [8]:
lst = lst_dat_region_score_import 
lst = lapply(lst, function(dat){
    tmp = any(is.na(dat))
    return(tmp)
})

print(lst)

$fcc_astarr_macs_input_overlap
[1] FALSE

$fcc_astarr_macs_input_union
[1] FALSE



## Sort regions by score

In [9]:
lst = lst_dat_region_score_import 
lol = lapply(lst, function(dat){
    ### foreach assay, sort regions by score
    lst = split(dat, dat$Assay_Label)
    lst = lapply(lst, function(dat){
        dat = dat %>% dplyr::arrange(desc(Score))
        vec = dat$Region
        return(vec)
    })
    return(lst)
})

### assign and show
lol_vec_txt_region_sorted = lol

lst = lol[[1]]
res = lapply(lst, head)
print(res)

$ASTARR
[1] "chr4:72112571-72112843"    "chr13:100392664-100393665"
[3] "chr14:29805715-29806017"   "chr16:89994836-89996150"  
[5] "chr2:189210051-189210466"  "chr8:90187578-90188106"   

$`CRISPRi-Growth`
[1] "chr11:33944532-33945260"   "chr11:118571936-118573512"
[3] "chr12:27022297-27023165"   "chr6:135322911-135323679" 
[5] "chr1:43358140-43359669"    "chr1:220093702-220094870" 

$`CRISPRi-HCRFF`
[1] "chr6:109303663-109305046"  "chr4:55635019-55636920"   
[3] "chr4:55637203-55637621"    "chr11:5284435-5285126"    
[5] "chr12:112012541-112014427" "chr11:5280584-5281266"    

$LMPRA
[1] "chr2:38643159-38644473"   "chr19:28606234-28607334" 
[3] "chr17:81512132-81514168"  "chr6:109482374-109483843"
[5] "chr7:97871705-97872960"   "chr7:44043962-44045299"  

$TMPRA
[1] "chr11:5591612-5591973"   "chr11:33161103-33161985"
[3] "chr2:60755781-60756991"  "chr11:34104955-34106361"
[5] "chr20:56828412-56828783" "chr11:5280584-5281266"  

$WSTARR
[1] "chr6:43627184-43627634"   "chr18:75712375-7

**Check: Region count**

In [10]:
lol = lol_vec_txt_region_sorted
lst = lapply(lol, function(lst){
    res = lapply(lst, length)
    dat = bind_rows(res)
    return(dat)
})

dat = bind_rows(lst, .id = "Folder")
fun_display_table(dat)

Folder,ASTARR,CRISPRi-Growth,CRISPRi-HCRFF,LMPRA,TMPRA,WSTARR
fcc_astarr_macs_input_overlap,150040,72743,925,61478,1148,146594
fcc_astarr_macs_input_union,246850,80288,1330,68497,1722,241031


## Apply robust aggregate ranks and calculate rank normalization scores

**Helper function**

In [11]:
fun_ranknorm = function(x) {
    ### if x runs 1, 2, ... , N, then
    ### we want (N - x + 1) / N  so that: rank = 1-> 1 & rank = N -> 0
    n = max(x, na.rm = TRUE) 
    r = n - x + 1
    return (r / n)
}

In [12]:
fun_mad = function(x){
    x_non_na = x[!is.na(x)]
    num = mad(
        x_non_na, 
        constant = 1, 
        na.rm    = TRUE, 
        center   = median(x_non_na)
    )
    return(num)
}


**Test code and matrix**

In [13]:
x = c("a", "y", "b", "x", "z")
y = c("x", "y", "z")
match(x, y)

[1] NA  2 NA  1  3

In [14]:
idx = "fcc_astarr_macs_input_overlap"
lol = lol_vec_txt_region_sorted
lst = lol[[idx]]

### apply robust aggregate ranks (RRA)
dat = aggregateRanks(lst, method = "RRA", full = TRUE)

### calculate raw-rank matrix
mat_num_rank = sapply(lst, function(x){match(dat$Name, x)})

In [15]:
fun_display_table(head(dat))

,Name,Score
chr19:47244024-47244799,chr19:47244024-47244799,0
chr13:97259755-97260733,chr13:97259755-97260733,0
chr1:118963502-118964080,chr1:118963502-118964080,0
chr16:87778744-87779533,chr16:87778744-87779533,0
chr3:126261107-126261399,chr3:126261107-126261399,0
chr9:116395345-116395887,chr9:116395345-116395887,0


In [16]:
fun_display_table(head(mat_num_rank))

ASTARR,CRISPRi-Growth,CRISPRi-HCRFF,LMPRA,TMPRA,WSTARR
14,24949,NA,11,NA,5
12,9487,NA,7,NA,36
74,NA,NA,16,NA,26
82,69245,NA,13,NA,20
190,NA,NA,42,NA,13
98,62152,NA,40,NA,109


In [17]:
### normalize so top = 1 and bottom = 0
mat_num_norm = apply(mat_num_rank, 2, function(vec){
    res = fun_ranknorm(vec)
    return(res)
})
fun_display_table(head(mat_num_norm))

ASTARR,CRISPRi-Growth,CRISPRi-HCRFF,LMPRA,TMPRA,WSTARR
0.9999134,0.6570392,NA,0.9998373,NA,0.9999727
0.9999267,0.8695957,NA,0.9999024,NA,0.9997612
0.9995135,NA,NA,0.9997560,NA,0.9998295
0.9994601,0.0481008,NA,0.9998048,NA,0.9998704
0.9987403,NA,NA,0.9993331,NA,0.9999181
0.9993535,0.1456085,NA,0.9993656,NA,0.9992633


In [18]:
vec = apply(mat_num_norm, 1, fun_mad)
head(vec)

[1] 6.768678e-05 8.272044e-05 7.345068e-05 2.051232e-04 5.850465e-04
[6] 5.117769e-05

**Apply RRA and Calculate RankNorm**

In [19]:
lol = lol_vec_txt_region_sorted
lst = lapply(lol, function(lst){
    ### apply robust aggregate ranks (RRA)
    dat = aggregateRanks(lst, method = "RRA", full = TRUE)
    
    ### calculate raw-rank matrix
    mat_num_rank = sapply(lst, function(x){match(dat$Name, x)})
    
    ### normalize so top = 1 and bottom = 0
    mat_num_norm = apply(mat_num_rank, 2, function(vec){
        res = fun_ranknorm(vec)
        return(res)
    })
    
    ### summarize scores and pvalues
    dat = dat %>%
        dplyr::mutate(
            RankNorm = rowMeans(mat_num_norm, na.rm = TRUE),
            RankMAD  = apply(mat_num_norm, 1, fun_mad),
            Padjust  = p.adjust(Score, method = "BH")
        )

    ### rename column
    dat = dat %>% 
        dplyr::rename(Region = Name, Pvalue = Score) %>%
        tidyr::separate(Region, into = c("Chrom", "ChromStart", "ChromEnd"), remove = FALSE) %>%
        dplyr::select(
            Chrom, ChromStart, ChromEnd, Region, 
            RankNorm, 
            RankMAD,
            Pvalue, 
            Padjust
        )
    
    return(dat)
})

### assign and show
lst_dat_region_rankagg = lst

res = lapply(lst, dim)
print(res)

dat = lst[[1]]
head(dat, 3)

$fcc_astarr_macs_input_overlap
[1] 150041      8

$fcc_astarr_macs_input_union
[1] 246851      8



,Chrom,ChromStart,ChromEnd,Region,RankNorm,RankMAD,Pvalue,Padjust
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr19:47244024-47244799,chr19,47244024,47244799,chr19:47244024-47244799,0.9141906,6.768678e-05,9.163905e-11,1.374961e-05
chr13:97259755-97260733,chr13,97259755,97260733,chr13:97259755-97260733,0.9672965,8.272044e-05,2.369185e-10,1.777374e-05
chr1:118963502-118964080,chr1,118963502,118964080,chr1:118963502-118964080,0.9996996,7.345068e-05,3.599111e-10,1.800047e-05


## Export results

In [20]:
for (txt_folder in VEC_TXT_FOLDER){
    ### get tables
    dat_region_score = lst_dat_region_rankagg[[txt_folder]]
    
    ### set file directory
    txt_fdiry  = file.path(FD_RES, "analysis_fcc_correlation", txt_folder)
    
    ### write table
    txt_fname = "region.coverage.ranknorm.aggregate.starrmpracrispr.tsv"
    txt_fpath = file.path(txt_fdiry, txt_fname)
    
    dat = dat_region_score
    dat = dat %>% dplyr::arrange(Chrom, ChromStart, ChromEnd)
    write_tsv(dat, txt_fpath)
}